In [12]:
"""
Script to extract text data from Mason petrology reports for load into SA_geodata. Input file path to the pdf 
documents and it will return an excel speadsheet containing the report no, sample no and description.
"""

'\nScript to extract text data from Mason petrology reports for load into SA_geodata. Input file path to the pdf \ndocuments and it will return an excel speadsheet containing the report no, sample no and description.\n'

In [13]:
import os
from pdf2image import convert_from_path, convert_from_bytes
import tempfile
import shutil
import re
import cv2 
import numpy as np
import pytesseract
from pytesseract import Output
import pandas as pd
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
from timeit import default_timer as timer
from datetime import datetime

In [14]:
#pdf_path = os.path.normpath(str(input(f'Please input the full path to the pdf files:')))
pdf_path = os.path.normpath('D:\\Python ML\\Mason-petrography-extractor\\Mason_reports\\test')

pdf_list = []
with os.scandir(pdf_path) as it:
    for entry in it:
        if entry.name.endswith(".pdf") and entry.is_file():
            pdf_list.append(entry.path)

print(pdf_list)

['D:\\Python ML\\Mason-petrography-extractor\\Mason_reports\\test\\MG4188.pdf', 'D:\\Python ML\\Mason-petrography-extractor\\Mason_reports\\test\\MG4196.pdf', 'D:\\Python ML\\Mason-petrography-extractor\\Mason_reports\\test\\MG4224.pdf']


In [15]:

start = timer()


report_dict = {'Report_No':[], 'Sample_No':[], 'Description':[]}
rock_type_dict = {'Sample_No':[], 'Rock_Type':[]}

# config for TesseractOCR
custom_config = r'--oem 3 --psm 6'

#create a temp directory and Convert pdf pages to individual jpg images for OCR and image extraction

temp_dir = tempfile.TemporaryDirectory()
f_name = ''


for pdf in pdf_list:
    jpg_path_list = []
    OCR_text_lst = []
    f_name = pdf.split('\\')[-1]
    images = convert_from_path(pdf, dpi = 300, output_folder=temp_dir.name,fmt='jpg')
    
# close PIL image objects
    for im in images:
        im.close()

    print(f'working on {f_name}')

#scan a file directory and return a list of jpg files to iterate over
    with os.scandir(temp_dir.name) as it:
        for entry in it:
            if entry.name.endswith(".jpg") and entry.is_file():
                jpg_path_list.append(entry.path)

    for im_path in jpg_path_list:
        image = cv2.imread(im_path, 0)
        d = pytesseract.image_to_string(image, config=custom_config)
        OCR_text_lst.append(d)

    #Grab the footer text from page 2 of pdf
    footer = OCR_text_lst[1].split('\n')
    footer[-1]

    #generate the regex pattern to remove the footer from pages
    start = footer[-1].split()[0] +' '+ footer[-1].split()[1]
    end = footer[-1].split()[-2] +' '+ footer[-1].split()[-1]
    footer_pattern = (f'{start}(.*?){end}')
    
    #generate a single string from the list of strings 
    text_string = ''
    raw_text = text_string.join(OCR_text_lst)
    raw_text

    #find and replace footer with white space
    text_c1 = re.sub(footer_pattern,'',raw_text)
    
    #Find the idx of the first instance of 'SAMPLE' used as a sample page headder, to  remove preceeding intro pages
    petro = text_c1.find('PETROGRAPHIC')
    first_sample = text_c1[petro:].find('SAMPLE') + petro
    
    # new string only contining the samples and removing all multiple newlines but adding extra newline at section headdings, then splitting into individual samples
    
    text_c2 = re.sub(r"\n+", "\n", text_c1[first_sample:])
    text_c2 = re.sub(r"(?<=\w)\.\n(?=\w)",".\n\n", text_c2)
    #text_c2 = re.sub(r"(?<=\w)\n(?=\w)"," ", text_c2,) #This will grab \n in text and replace w wht space but messes with mineral lists

    text_c2 = text_c2.replace('SAMPLE','***SAMPLE').replace('HOLE','\nHOLE').replace('LITH','\nLITH').replace('COMMENT','\nCOMMENT').replace('SECTION','\nSECTION').replace('HAND','\nHAND').replace('FIG','\nFIG').replace('ROCK','\nROCK').replace('PETROGRAPHY','\nPETROGRAPHY').replace('Mineral','\nMineral').replace('In thin section','\nIn thin section').replace('INTERPRETATION','\nINTERPRETATION').replace('SOURCE','\nSOURCE').replace('OTHER','\nOTHER').replace('STRAT/\nLITH', '\nSTRAT/LITH').replace('APPENDIX','\nAPPENDIX').replace('BRIEF','\nBRIEF').replace('In polished thin','\nIn polished thin').replace('CLIENT','\nCLIENT').replace('FIELD','\nFIELD').replace('GA NO','\nGA NO').split('***')

   #remove empty strings from list
    descriptions = [x for x in text_c2 if x]
    
    fig_pattern = '(FIG|Fig)'
    sample_no_list =[]
    sample_no_pattern = '(\d{7})|(R\d{7})'
    fig_list = []
    rock_pattern = r"(?s)ROCK N.*?:(.*?)PETRO"
    rock_list = []
    for l in descriptions:
        m = re.search(sample_no_pattern,l)
        f = re.search(fig_pattern,l)
        roc = re.search(rock_pattern,l,)
        if m:
            sample_no_list.append(m.group().strip('R'))
        if f:
            fig_list.append(m.group().strip('R'))
        if roc:
            rock_list.append(roc.group(1))

    print(f'lengths are equal :{len(descriptions) == len(sample_no_list)}')

    report_list = []
    for i in range(len(descriptions)):
        report_list.append(f_name)

    report_dict['Report_No'].append(report_list)
    report_dict['Sample_No'].append(sample_no_list)
    report_dict['Description'].append(descriptions)
    rock_type_dict['Sample_No'].append(sample_no_list)
    rock_type_dict['Rock_Type'].append(rock_list)

    #remove images from previous loop
    with os.scandir(temp_dir.name) as it:
        for entry in it:
            if entry.name.endswith(".jpg") and entry.is_file():
                os.remove(entry)



working on MG4188.pdf
lengths are equal :True
working on MG4196.pdf
lengths are equal :True
working on MG4224.pdf
lengths are equal :True


In [16]:
#print(text_c2)

In [17]:
# Cell to verify all reports have correct number of samples and descriptions, if not they can be dropped from export
length_chek = []
index_list =[]
for i, (a,b,c,l,k) in enumerate(zip(report_dict['Report_No'],report_dict['Sample_No'],report_dict['Description'],rock_type_dict['Sample_No'],rock_type_dict['Rock_Type'])):
    length_chek.append([a[0],len(a),len(b),len(c),len(l),len(k)])
    if len(b)!=len(c) or len(l)!=len(k):
        index_list.append(i)


length_chek

[['MG4188.pdf', 32, 32, 32, 32, 32],
 ['MG4196.pdf', 4, 4, 4, 4, 4],
 ['MG4224.pdf', 67, 67, 67, 67, 67]]

In [18]:
index_list

[]

In [19]:
df_all = pd.DataFrame(report_dict)
df_all

,Report_No,Sample_No,Description
0,"[MG4188.pdf, MG4188.pdf, MG4188.pdf, MG4188.pd...","[2137948, 2137949, 2137987, 2138095, 2138096, ...","[SAMPLE ID : R2137948 (Wudinna region, South A..."
1,"[MG4196.pdf, MG4196.pdf, MG4196.pdf, MG4196.pdf]","[1588243, 1588242, 1588240, 1588241]","[SAMPLE R# : R1588243 (Barossa Complex, South ..."
2,"[MG4224.pdf, MG4224.pdf, MG4224.pdf, MG4224.pd...","[2427063, 2427064, 2427070, 2427072, 2427074, ...","[SAMPLE : 2427063 (Coompana Drilling Project, ..."


In [20]:
df_failed = df_all.loc[index_list]

In [21]:
# create dataframe and export to excel
df = pd.DataFrame(report_dict).drop(index_list).apply(pd.Series.explode)

df2 = pd.Series(fig_list)

df_failed = df_all.loc[index_list]

df_rock = pd.DataFrame(rock_type_dict).drop(index_list).apply(pd.Series.explode)

now = datetime.now().strftime("%Y%m%d_%H%M")

with pd.ExcelWriter(pdf_path+'\\'+now+'_Mason_petrology_export.xlsx') as writer:
    df.to_excel(writer, sheet_name = 'extracted_pet_descriptions', index=False)
    df_failed.to_excel(writer, sheet_name = 'Failed_extract', index=False)
    df2.to_excel(writer, sheet_name='Samples with figs', index=False)

with pd.ExcelWriter(pdf_path+'\\'+now+'_Rock_types.xlsx') as writer:
    df_rock.to_excel(writer, sheet_name = 'Mason_rock_types', index=False)




C:\Users\dutchr01\AppData\Local\Continuum\anaconda3\envs\tesseractOCR\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


In [22]:
fig_list

[]